## Import code

In [1]:
%load_ext autoreload
%autoreload 2 
# Autoreloads packages when changes are detected

from src.Data import * # Class containing the data
from src.Assignment import * # Class containing an assignment
from src.Model import * # Class containing a Pulp model used for optimization
from src.ModelColumnGen import * # Class containing Pulp model that optimizes using column generation
from src.ModelFracStable import * # Class containing a Pulp model for finding an fractionally stable stochastic improvement
from src.ModelHeuristicLP import * # Class containing heuristic that will use generated weakly stable matchings as an input
from src.DataGen import * # Generate student preferences and school priorities
from src.DA_STB import * # Generate DA assignment with single tie-breaking (STB)

# If you get error that pulp and gurobipy are not installed: uncomment following lines in src/Data file:(keep exclamation marks)
    #! pip install pulp
    #! pip install gurobipy

## Column generation formulation

In [ ]:
# Generate random data
parameters = DataGenParam(mean_pref = 4, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
[n_students,n_schools,seed] = [8,4,0]
name = str(n_students) + "_" + str(n_schools) + "_" + str(seed)
MyData = generate_data(n_students = n_students, n_schools=n_schools, parameters = parameters, name=name, print_data=False, seed = seed)

In [34]:
# Print data if desired
print(MyData)

The data instance has the following properties: 

	300 students.
	50 schools. 

 	PREFERENCES:
	0	9 29 46 36 
	1	47 28 46 39 36 
	2	42 29 17 
	3	26 12 
	4	38 9 28 26 
	5	32 37 
	6	6 30 21 
	7	29 35 27 37 
	8	25 
	9	22 15 12 36 
	10	35 11 46 
	11	32 25 41 22 
	12	23 14 7 
	13	35 41 30 17 
	14	28 27 45 17 
	15	9 41 43 
	16	2 12 31 39 29 
	17	11 6 25 39 
	18	12 3 28 13 
	19	3 38 40 20 16 
	20	15 18 29 26 
	21	17 31 35 27 
	22	19 29 28 16 
	23	16 27 
	24	29 28 17 
	25	28 47 21 
	26	41 4 
	27	41 45 13 0 
	28	24 10 46 
	29	43 46 
	30	29 24 
	31	40 30 13 45 
	32	22 38 24 9 
	33	16 29 7 0 22 
	34	37 29 16 
	35	36 23 49 38 40 
	36	14 7 12 24 28 
	37	12 39 41 16 24 
	38	29 
	39	36 7 16 37 27 
	40	43 22 20 16 
	41	12 32 16 41 
	42	15 33 28 30 
	43	43 9 7 6 
	44	4 27 9 39 
	45	0 20 24 
	46	12 18 
	47	12 16 45 
	48	21 18 30 
	49	32 31 5 28 37 
	50	39 27 6 
	51	5 17 45 24 
	52	43 22 37 
	53	24 36 16 
	54	42 14 0 
	55	36 31 
	56	31 35 27 6 
	57	32 0 12 
	58	21 18 
	59	6 
	60	21 30 17 36 
	61	12 7 37 

In [35]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

Students in ties: 300
Tie-breaking rules needed: 306057512216440636035370461297268629388588804173576999416776741259476533176716867465515291422477573349939147888701726368864263907759003154226842927906974559841225476930271954604008012215776252176854255965356903506788725264321896264299365204576448830388909753943489625436053225980776521270822437639449120128678675368305712293681943649956460498166450227716500185176546469340112226034729724066333258583506870150169794168850353752137554910289126407157154830282284937952636580145235233156936482233436799254594095276820608062232812387383880817049600000000000000000000000000000000000000000000000000000000000000000000000000
Tie-breaking rules sampled: 1000


100%|██████████| 1000/1000 [04:10<00:00,  4.00it/s]


[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 1.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.238 0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]]


In [37]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModel = ModelColumnGen(MyData, A, True)
q = MyModel.Solve("TRAD", "GUROBI", print_log=False, print_out=False)
#q = MyModel.Solve("STABLE", "GUROBI", True)


Average rank before optimization: 1.3781300000000025.


Number of matchings: 1000
ITERATION: 1
Objective master:  1.3504003998079148
		Objective pricing:  26.727372415567086
Optimal solution found! Best average rank:  1.3504003998079148
Original average rank:  1.3781300000000025


In [13]:
print(q.assignment)
print(MyModel.Xdecomp)
print(MyModel.Xdecomp_coeff)

[[0.    0.    0.    0.    0.    0.    0.    0.    0.494 0.   ]
 [0.    0.    0.    0.    0.    0.494 0.    0.    0.506 0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.    1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.507 0.    0.    0.493]
 [0.    0.    0.    0.    0.    0.    0.    0.493 0.    0.507]
 [0.511 0.041 0.    0.    0.    0.244 0.    0.    0.    0.   ]
 [0.    0.379 0.    0.    0.    0.183 0.    0.    0.    0.   ]
 [0.489 0.    0.    0.    0.    0.079 0.    0.    0.    0.   ]
 [0.    0.58  0.    0.    0.    0.    0.    0.42  0.    0.   ]]
[array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

## Run initial IP on generated data
Specify the number of students and schools, and run the models for this data.

In [ ]:
# Generate random data
parameters = DataGenParam(mean_pref = 6, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
MyData = generate_data(n_students=5, n_schools=4, parameters = parameters, name="Test_DataGen", print_data=False, seed = 0)

In [ ]:
# Print data if desired
print(MyData)

In [ ]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - A.assignment, "TestDataGen_Diff")
diff.visualize()

## Heuristic subset weakly stable matchings
Runs an LP that minimizes the average rank while stochastically dominating the DA assignment with single-tie breaking, and while only using matchings that were used to compute DA probabilities. This is a heuristic.

In [ ]:
# Generate random data
parameters = DataGenParam(mean_pref = 5, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
MyData = generate_data(n_students=10, n_schools=4, parameters = parameters, name="500_25", print_data=False, seed = 1)

In [ ]:
# Print data if desired
print(MyData)

In [ ]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModel = ModelHeuristicLP(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
print(np.max(q.assignment-A.assignment))

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - A.assignment, "40_12_Diff")
diff.visualize()

## Heuristic Fractional stable random matching

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModelFS = ModelFracStable(MyData, A, True)
q = MyModelFS.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# FInd decomposition over weakly stable matchings of this fractionally stable matching (if decomposition exists)
MyModelCHECK = Model(MyData, q, True)
q_check = MyModelCHECK.Solve("STABLE", "GUROBI", True)

## Run code manual data
Manually enter data.

In [ ]:
# Define preferences of the students
# 'pref[i][k]' contains the position of the k-th ranked school in the preferences.
# We assume the preferences to be strict
# Note that preferences can be strict. We indicate this by a tuple () in the list.

# Example paper
n_stud = 4
n_schools = 4

file_name = "Ex_paper"

# Preferences students
pref = [['1', '3', '4', '2'],
       ['1','4','3','2'],
       # ['1', '4'],
       ['2','3', '4', '1'],
       ['2', '4', '3', '1']]

# Priorities schools
prior = [[('A', 'B'), 'C', 'D'],
        [('C', 'D'), 'A', 'B'],
        ['B', 'D', ('A', 'C')],
        ['A', 'C', ('B', 'D')]]


# Capacities schools
cap = [1,1,1,1]

# Names of students and schools
ID_stud = ["A", "B", "C", "D"]
ID_school = ["1", "2", "3", "4"]

# Also create the random matching upon which we want to improve
p = np.zeros(shape=(n_stud, n_schools))
p[0][0] = 1/2
p[1][0] = 1/2
p[2][1] = 1/2
p[3][1] = 1/2
p[0][2] = 3/8
p[2][2] = 3/8
p[1][3] = 3/8
p[3][3] = 3/8
p[0][3] = 1/8
p[2][3] = 1/8
p[1][2] = 1/8
p[3][2] = 1/8

In [ ]:
# Generate a data instance (and print it)
MyData = Data(n_stud, n_schools, pref, prior, cap, ID_stud, ID_school, file_name)
print(MyData)

In [ ]:
# Generate an Assignment instance (and visualize it)
A = Assignment(MyData, p, "Ex_paper")

# To visualize assignment
A.visualize()

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, False)
q = MyModel.Solve("IMPR_RANK", "GUROBI", False)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - p, "Ex_paper_Diff")
diff.visualize()

In [ ]:
tuple([1,2])